In [1]:
#After adding all the features we iproved from R^2 = 0.43 to R^2 = 0.49

In [2]:
# imports 
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
pd.set_option('display.float_format', '{:.2f}'.format)


import matplotlib.pyplot as plt
%matplotlib inline  


In [3]:
#reading the csv and select to columns from the modle
Anime_df = pd.read_csv("Anime_df_Full_Floats.csv")
Anime_df["Genres"] = Anime_df["Genres"].str.replace(", ",",")
Anime_df_Top_500 = Anime_df.head(500)
Anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5559 entries, 0 to 5558
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Anime_Name  5559 non-null   object 
 1   Score       5559 non-null   float64
 2   Rank        5559 non-null   float64
 3   Scored_By   5559 non-null   float64
 4   Episodes    5559 non-null   float64
 5   Duration    5559 non-null   float64
 6   Popularity  5559 non-null   float64
 7   Members     5559 non-null   float64
 8   Favorites   5559 non-null   float64
 9   Status      5559 non-null   object 
 10  Genres      5559 non-null   object 
 11  Rating      5559 non-null   object 
 12  Source      5559 non-null   object 
 13  Aired       5559 non-null   object 
 14  Producers   5559 non-null   object 
 15  Studios     5559 non-null   object 
 16  Premiered   5559 non-null   object 
 17  Broadcast   5559 non-null   object 
 18  Licensors   5559 non-null   object 
dtypes: float64(8), object(11)
m

In [4]:
len(Anime_df.Premiered.unique())

235

In [5]:
#selecting the columns from the modle
#low corr
Train_Columns = ["Score",
"Rank",
"Scored_By",
"Episodes",
"Duration",
"Popularity",
"Members",
"Favorites"]
Train_df = Anime_df[Train_Columns]##with this DF we got R^2 = 0.46
Train_df.corr(method = "pearson").Score.sort_values(ascending = False)

Score         1.00
Members       0.46
Scored_By     0.42
Favorites     0.34
Duration      0.27
Episodes     -0.03
Popularity   -0.31
Rank         -0.51
Name: Score, dtype: float64

In [6]:
#with the highest corr
Train_df_2 = Train_df.drop(["Duration", "Episodes", "Popularity"], axis = 1)#with less columns we got R^2 = 0.43
Train_df_2.corr(method = "pearson").Score.sort_values(ascending = False)
Train_df_2

,Score,Rank,Scored_By,Members,Favorites
0,9.15,1.00,1674539.00,2753439.00,193718.00
1,9.09,2.00,175304.00,502351.00,13321.00
2,9.09,3.00,1142680.00,1683363.00,47486.00
3,9.09,4.00,1131835.00,2144169.00,164108.00
4,9.06,5.00,128247.00,302846.00,13965.00
...,...,...,...,...,...
5554,6.86,17408.00,69593.15,246.00,1.00
5555,6.86,17419.00,69593.15,45.00,0.00
5556,6.86,17421.00,69593.15,410.00,0.00
5557,6.86,17423.00,69593.15,605.00,1.00


In [7]:
Anime_Genres_df_dumm = pd.concat([Anime_df["Genres"].str.get_dummies(sep=",")])
Anime_Genres_df_dumm.sum().sort_values(ascending = False)

Comedy           2513
Action           1512
Adventure        1239
Fantasy          1152
Sci-Fi            979
Drama             935
Slice of Life     888
Romance           837
Supernatural      608
Unknown           556
Sports            330
Mystery           327
Ecchi             305
Horror            143
Gourmet            58
Suspense           57
Girls Love         42
Boys Love          37
Erotica            24
Avant Garde        19
Work Life           4
dtype: int64

In [8]:
#dropping genres with low number of animes
Anime_Genres_df_dumm = Anime_Genres_df_dumm.drop(["Sports",
"Mystery",
"Ecchi",
"Horror",
"Gourmet",
"Suspense",
"Girls Love",
"Boys Love",
"Erotica",
"Avant Garde",
"Work Life"],axis = 1)
Anime_Genres_df_dumm

,Action,Adventure,Comedy,Drama,Fantasy,Romance,Sci-Fi,Slice of Life,Supernatural,Unknown
0,1,1,1,1,1,0,0,0,0,0
1,1,0,1,0,0,0,1,0,0,0
2,1,0,0,1,1,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0
4,0,0,1,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
5554,0,0,1,0,0,0,0,0,0,0
5555,0,0,1,0,0,0,1,0,0,0
5556,0,0,1,0,0,0,1,0,0,0
5557,1,0,0,0,0,0,0,0,0,0


In [9]:
source_dumm_df = pd.concat([Anime_df["Source"].str.get_dummies(sep=",")])
Rating_dumm_df = pd.concat([Anime_df["Rating"].str.get_dummies(sep=",")])

Train_df_source_with_dumm = pd.concat([Train_df,Rating_dumm_df,source_dumm_df,Anime_Genres_df_dumm],axis = 1)
Train_df_source_with_dumm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5559 entries, 0 to 5558
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Score                           5559 non-null   float64
 1   Rank                            5559 non-null   float64
 2   Scored_By                       5559 non-null   float64
 3   Episodes                        5559 non-null   float64
 4   Duration                        5559 non-null   float64
 5   Popularity                      5559 non-null   float64
 6   Members                         5559 non-null   float64
 7   Favorites                       5559 non-null   float64
 8   G - All Ages                    5559 non-null   int64  
 9   None                            5559 non-null   int64  
 10  PG - Children                   5559 non-null   int64  
 11  PG-13 - Teens 13 or older       5559 non-null   int64  
 12  R - 17+ (violence & profanity)  55

In [10]:
Train_df_source_with_dumm.to_csv("ML_Df.csv", index = False)

In [11]:
df_ID = Anime_df['Anime_Name']
TRAINING_FEATURES = Train_df_source_with_dumm[Train_df_source_with_dumm.columns.difference(['Score'])]

TARGET_FEATURE    = 'Score'

X = TRAINING_FEATURES
y = Train_df_source_with_dumm[TARGET_FEATURE]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,4-koma manga,Action,Adventure,Book,Card game,Comedy,Digital manga,Drama,Duration,Episodes,...,Rank,Romance,Sci-Fi,Scored_By,Slice of Life,Supernatural,Unknown,Unknown,Visual novel,Web manga
1392,0,0,0,0,0,1,0,0,24.00,12.00,...,2881.00,1,0,106439.00,0,0,0,0,0,0
2633,0,0,0,0,0,1,0,0,23.00,13.00,...,5891.00,0,0,6574.00,1,0,0,0,0,1
2574,0,0,1,0,1,0,0,0,25.00,12.00,...,5716.00,0,0,592.00,0,0,0,0,0,0
1255,0,0,1,0,0,1,0,0,24.00,50.00,...,2582.00,1,0,40696.00,0,0,0,0,0,0
4228,0,1,0,0,0,0,0,0,3.00,10.00,...,11717.00,0,0,1891.00,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0,1,0,0,0,0,0,0,11.00,10.00,...,8749.00,0,1,184.00,0,0,0,0,0,0
5191,0,1,0,0,0,0,0,0,24.00,30.00,...,15842.00,0,1,69593.15,0,0,0,0,0,0
5226,0,1,0,0,0,1,0,0,25.00,260.00,...,15955.00,0,0,69593.15,0,0,0,0,0,0
5390,0,0,0,0,0,1,0,1,25.00,208.00,...,16701.00,0,0,69593.15,0,0,0,0,0,0


In [13]:
print("Initial amount of samples: #{}".format(X.shape))
print("Number of training samples: #{}".format(X_train.shape))
print("Number of test samples: #{}".format(X_test.shape))

print("\nTarget distribution in original dataset:\n{}".format(y.value_counts()))
print("\nTarget distribution in the training set:\n{}\n".format(y_train.value_counts()))
print("Target distribution in the test set:\n{}".format(y_test.value_counts()))

Initial amount of samples: #(5559, 40)
Number of training samples: #(4447, 40)
Number of test samples: #(1112, 40)

Target distribution in original dataset:
6.86    1449
7.21      34
6.54      31
7.02      29
7.16      29
        ... 
4.91       1
4.87       1
4.85       1
4.84       1
2.95       1
Name: Score, Length: 418, dtype: int64

Target distribution in the training set:
6.86    1155
7.21      29
7.27      24
7.16      22
6.67      22
        ... 
9.15       1
4.75       1
9.05       1
5.14       1
8.54       1
Name: Score, Length: 404, dtype: int64

Target distribution in the test set:
6.86    294
6.54     10
6.81      9
6.50      8
6.13      8
       ... 
7.32      1
4.39      1
8.02      1
7.00      1
5.58      1
Name: Score, Length: 299, dtype: int64


In [14]:
model = linear_model.LinearRegression()

In [15]:
model.fit(X_train,y_train)

LinearRegression()

In [16]:
y_pred = model.predict(X_test)

In [17]:
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred))

Coefficients: [ 2.12968443e+04 -6.89456788e-02  1.90643205e-02  2.12968405e+04
  2.12966824e+04  3.72669476e-02  2.12963446e+04  1.37282353e-01
  2.41702120e-03  1.38110892e-04 -9.43863205e-03 -1.17933540e-06
  1.27534033e-02  2.12968293e+04  2.12970561e+04  2.12970680e+04
 -2.96352249e-06  2.12966836e+04  2.12964683e+04 -5.03922703e-02
  2.12969138e+04  2.12969625e+04  2.12968957e+04  8.95644923e-02
  6.90112391e-03  2.12969974e+04  2.90971942e-05  1.45764292e-01
 -2.04591709e-01  2.12971905e+04 -1.20258009e-04  1.14505420e-03
  1.53790016e-02  6.34685321e-06  1.17553136e-01  7.22749544e-02
  2.12969623e+04  2.02946891e-01  2.12968846e+04  2.12968974e+04]
Intercept: -21289.759149270252
Mean squared error (MSE): 0.24
Coefficient of determination (R^2): 0.49


In [18]:
r2_score(y_test, y_pred)

0.4940097509870911

In [19]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_trained_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [20]:
#Pearson
Train_df.corr(method = "pearson").Score.sort_values(ascending = False)

Score         1.00
Members       0.46
Scored_By     0.42
Favorites     0.34
Duration      0.27
Episodes     -0.03
Popularity   -0.31
Rank         -0.51
Name: Score, dtype: float64

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Initial amount of samples: #{}".format(X.shape))
print("Number of training samples: #{}".format(X_train.shape))
print("Number of test samples: #{}".format(X_test.shape))

print("\nTarget distribution in original dataset:\n{}".format(y.value_counts()))
print("\nTarget distribution in the training set:\n{}\n".format(y_train.value_counts()))
print("Target distribution in the test set:\n{}".format(y_test.value_counts()))

Initial amount of samples: #(5559, 40)
Number of training samples: #(4447, 40)
Number of test samples: #(1112, 40)

Target distribution in original dataset:
6.86    1449
7.21      34
6.54      31
7.02      29
7.16      29
        ... 
4.91       1
4.87       1
4.85       1
4.84       1
2.95       1
Name: Score, Length: 418, dtype: int64

Target distribution in the training set:
6.86    1155
7.21      29
7.27      24
7.16      22
6.67      22
        ... 
9.15       1
4.75       1
9.05       1
5.14       1
8.54       1
Name: Score, Length: 404, dtype: int64

Target distribution in the test set:
6.86    294
6.54     10
6.81      9
6.50      8
6.13      8
       ... 
7.32      1
4.39      1
8.02      1
7.00      1
5.58      1
Name: Score, Length: 299, dtype: int64


In [22]:
model = linear_model.LinearRegression().fit(X_train,y_train)

In [23]:
model.fit(X_train,y_train)

LinearRegression()

In [24]:
y_pred = model.predict(X_test)
y_pred

array([6.3988375 , 6.43937333, 6.45850665, ..., 7.04578214, 6.65023822,
       7.46232999])

In [25]:
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred))
print('model score is (R^2): %.2f'
      % r2_score(y_test, y_pred))

Intercept: -21289.759149270252
Mean squared error (MSE): 0.24
model score is (R^2): 0.49


In [26]:
r2_score(y_test, y_pred)

0.4940097509870911

In [27]:
#scaling features
scaler = MinMaxScaler()
scaler.fit(X_train)

X_trained_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [28]:
X_trained_scaled = pd.DataFrame(X_trained_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns= X_test.columns)

In [29]:
np.round(X_trained_scaled.describe(),2)

,4-koma manga,Action,Adventure,Book,Card game,Comedy,Digital manga,Drama,Duration,Episodes,...,Rank,Romance,Sci-Fi,Scored_By,Slice of Life,Supernatural,Unknown,Unknown,Visual novel,Web manga
count,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,...,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00,4447.00
mean,0.03,0.27,0.22,0.01,0.01,0.45,0.00,0.17,0.33,0.01,...,0.41,0.15,0.17,0.03,0.16,0.11,0.15,0.10,0.03,0.02
std,0.17,0.44,0.41,0.09,0.09,0.50,0.04,0.37,0.15,0.03,...,0.29,0.35,0.38,0.06,0.37,0.31,0.36,0.30,0.17,0.13
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,...,0.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.40,0.01,...,0.37,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00
75%,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.42,0.01,...,0.63,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [30]:
model_scaled = LinearRegression().fit(X_trained_scaled, y_train)

In [31]:
print('Coefficients:', model_scaled.coef_)
print('Intercept:', model_scaled.intercept_)
print('model score is',model_scaled.score(X_trained_scaled,y_train))

Coefficients: [-2.85930294e+12 -6.69138616e-02  1.78500977e-02 -2.85930294e+12
 -2.85930294e+12  3.89020745e-02 -2.85930294e+12  1.34756124e-01
  1.35199632e-01  4.13191225e-01 -1.05141834e-02 -2.35174431e-01
  1.53139295e-02 -2.85930294e+12 -2.85930294e+12 -2.85930294e+12
 -9.62161685e+00 -2.85930294e+12 -2.85930294e+12 -4.74800134e-02
 -2.85930294e+12 -2.85930294e+12 -2.85930294e+12  9.19673923e-02
  9.37355050e-03 -2.85930294e+12  5.61304835e-01  1.50616064e-01
 -2.00230345e-01 -2.85930294e+12 -2.09523335e+00  1.58200450e-03
  1.56309022e-02  1.42064927e+01  1.17400062e-01  7.21207751e-02
 -2.85930294e+12  2.04314435e-01 -2.85930294e+12 -2.85930294e+12]
Intercept: 2859302936605.143
model score is 0.5115865839341449


In [32]:
y_pred_scaled = model_scaled.predict(X_test_scaled)

In [33]:
y_pred_scaled

array([6.42382812, 6.44580078, 6.47460938, ..., 7.04833984, 6.66259766,
       7.45703125])

In [34]:
print('mean squared error on scaled data is ', np.round(mean_squared_error(y_pred_scaled, y_test)),2)
print('r squared on scaled data is ', r2_score(y_pred_scaled, y_test))

mean squared error on scaled data is  0.0 2
r squared on scaled data is  0.09966732181957727
